In [5]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import statsmodels.api as sm
import statistics as st
import requests
import re

## Download Data from Kenneth French Data Library

In [22]:
ff = pdr.get_data_famafrench('F-F_Research_Data_Factors_daily', '2019-01-01','2019-12-31')[0]
ff = ff.rename({'RF': 'Risk-Free'}, axis=1)
mom = pdr.get_data_famafrench('F-F_Momentum_Factor_daily', '2019-01-01','2019-12-31')[0]
ff = ff.join(mom)

##  Web Scraping Using Regular Expression

Use regular expression to extract complete list of banks stock trading on the NYSE from topforeignstocks: https://topforeignstocks.com/stock-lists/the-complete-list-of-bank-stocks-trading-on-the-nyse-2/.

In [3]:
url = 'https://topforeignstocks.com/stock-lists/the-complete-list-of-bank-stocks-trading-on-the-nyse-2/'
req = requests.get(url)
result = req.text

In [4]:
chunks = re.findall(r'<td class="column-1">(.*?)<tr class="', req.text, re.DOTALL)
len(chunks)

92

In [6]:
from pandas import DataFrame

data = []
comps = []
for chunk in chunks:
    bk_name = re.search(r'<td class="column-2">(.*?)</td>', chunk).group(1)
    bk_ticker = re.search(r'>([A-Z]+)<\/a><\/td>', chunk).group(1)
    data.append((bk_name, bk_ticker))
    comps.append(bk_ticker)
    df_data = DataFrame(data,columns=['name','ticker'])

In [7]:
df_data.head()

,name,ticker
0,Ally Financial Inc.,ALLY
1,Associated Banc-Corp,ASB
2,"Axos Financial, Inc.",AX
3,"Banc of California, Inc.",BANC
4,BancorpSouth Bank,BXS


## CAPM Model - Beta 

In [8]:
# 2019
prcs = pdr.get_data_yahoo(comps, '2019-01-01','2019-12-31')
idx = pd.IndexSlice
prcs = prcs.loc[:, idx['Adj Close', :]]
prcs.columns = prcs.columns.levels[1]
prcs = prcs.dropna(axis=1, how='all')
new_comps = prcs.columns.tolist()
len(new_comps)

/Users/anyidu/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/base.py:270: SymbolWarning: Failed to read symbol: 'BBT', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/anyidu/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/base.py:270: SymbolWarning: Failed to read symbol: 'FCB', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/anyidu/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/base.py:270: SymbolWarning: Failed to read symbol: 'MFCB', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/anyidu/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/base.py:270: SymbolWarning: Failed to read symbol: 'MSL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/anyidu/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/base.py:270: SymbolWarning: Failed to read symbol: 'BFR', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)

85

In [30]:
rets = prcs.pct_change()
df = ff.join(rets, how='left')
df[new_comps] *= 100
df = df.dropna(axis=0, how='any')

In [31]:
beta=[]
# CAPM: one-factor model
for comp in new_comps:
    print(f'Company: {comp}\n')
    reg = sm.OLS(df[comp] - df['Risk-Free'], df['Mkt-RF']).fit()
    beta.append(reg.params)
    print(reg.summary2())
    print(f'Beta/Systematic Risk of {comp} is % .4f \n'
                % (reg.params)) 
    print(f'Stock volatility of {comp} is % .4f \n'
                % (st.stdev(df[comp]*252**0.5))) 
    annre = (prcs[comp][-1]-prcs[comp][0])/(prcs[comp][0])
    print(f'Annual return of {comp} is % .4f \n'
                % (annre*100))


Company: ALLY

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.486   
Dependent Variable:     y                AIC:                         530.2613
Date:                   2020-05-28 21:58 BIC:                         533.4261
No. Observations:       175              Log-Likelihood:              -264.13 
Df Model:               1                F-statistic:                 166.5   
Df Residuals:           174              Prob (F-statistic):          3.71e-27
R-squared (uncentered): 0.489            Scale:                       1.2050  
-----------------------------------------------------------------------------------
              Coef.       Std.Err.         t         P>|t|       [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF        1.3051        0.1011      12.9028      0.0000      1.1054      1.5047
-----------------------------------------


Beta/Systematic Risk of BOH is  0.9638 

Stock volatility of BOH is  18.1312 

Annual return of BOH is  43.5380 

Company: BK

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.314   
Dependent Variable:     y                AIC:                         535.7644
Date:                   2020-05-28 21:58 BIC:                         538.9292
No. Observations:       175              Log-Likelihood:              -266.88 
Df Model:               1                F-statistic:                 81.03   
Df Residuals:           174              Prob (F-statistic):          3.80e-16
R-squared (uncentered): 0.318            Scale:                       1.2435  
-----------------------------------------------------------------------------------
               Coef.       Std.Err.        t         P>|t|       [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF      

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.340   
Dependent Variable:     y                AIC:                         639.3027
Date:                   2020-05-28 21:58 BIC:                         642.4675
No. Observations:       175              Log-Likelihood:              -318.65 
Df Model:               1                F-statistic:                 91.16   
Df Residuals:           174              Prob (F-statistic):          1.23e-17
R-squared (uncentered): 0.344            Scale:                       2.2470  
-----------------------------------------------------------------------------------
               Coef.       Std.Err.        t         P>|t|       [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF         1.3187        0.1381      9.5476      0.0000      1.0461      1.5913
--------------------------------------------------------

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.355   
Dependent Variable:     y                AIC:                         534.3689
Date:                   2020-05-28 21:58 BIC:                         537.5337
No. Observations:       175              Log-Likelihood:              -266.18 
Df Model:               1                F-statistic:                 97.45   
Df Residuals:           174              Prob (F-statistic):          1.57e-18
R-squared (uncentered): 0.359            Scale:                       1.2336  
-----------------------------------------------------------------------------------
               Coef.       Std.Err.        t         P>|t|       [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF         1.0103        0.1023      9.8717      0.0000      0.8083      1.2123
--------------------------------------------------------

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.416   
Dependent Variable:     y                AIC:                         634.6281
Date:                   2020-05-28 21:58 BIC:                         637.7929
No. Observations:       175              Log-Likelihood:              -316.31 
Df Model:               1                F-statistic:                 125.6   
Df Residuals:           174              Prob (F-statistic):          2.71e-22
R-squared (uncentered): 0.419            Scale:                       2.1877  
-----------------------------------------------------------------------------------
              Coef.       Std.Err.         t         P>|t|       [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF        1.5275        0.1363      11.2078      0.0000      1.2585      1.7965
--------------------------------------------------------


Beta/Systematic Risk of BCH is  0.6837 

Stock volatility of BCH is  30.5187 

Annual return of BCH is -25.0524 

Company: BSAC

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.113   
Dependent Variable:     y                AIC:                         670.8896
Date:                   2020-05-28 21:58 BIC:                         674.0544
No. Observations:       175              Log-Likelihood:              -334.44 
Df Model:               1                F-statistic:                 23.37   
Df Residuals:           174              Prob (F-statistic):          2.93e-06
R-squared (uncentered): 0.118            Scale:                       2.6914  
-----------------------------------------------------------------------------------
               Coef.       Std.Err.        t         P>|t|       [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF    


Beta/Systematic Risk of OFG is  1.3908 

Stock volatility of OFG is  35.9941 

Annual return of OFG is  43.8507 

Company: SMFG

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.397   
Dependent Variable:     y                AIC:                         370.5633
Date:                   2020-05-28 21:58 BIC:                         373.7281
No. Observations:       175              Log-Likelihood:              -184.28 
Df Model:               1                F-statistic:                 116.1   
Df Residuals:           174              Prob (F-statistic):          4.65e-21
R-squared (uncentered): 0.400            Scale:                       0.48381 
-----------------------------------------------------------------------------------
              Coef.       Std.Err.         t         P>|t|       [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF    


Beta/Systematic Risk of ING is  1.1389 

Stock volatility of ING is  23.2127 

Annual return of ING is  20.7540 

Company: BCS

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.273   
Dependent Variable:     y                AIC:                         639.1223
Date:                   2020-05-28 21:58 BIC:                         642.2871
No. Observations:       175              Log-Likelihood:              -318.56 
Df Model:               1                F-statistic:                 66.76   
Df Residuals:           174              Prob (F-statistic):          6.08e-14
R-squared (uncentered): 0.277            Scale:                       2.2447  
-----------------------------------------------------------------------------------
               Coef.       Std.Err.        t         P>|t|       [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF     

## Fama-French Four Model: Beta, SMB, HML, Mom

In [40]:
# Fama-French 4 model
params = []
for comp in new_comps:
    print(f'Company: {comp}\n')
    reg2 = sm.OLS(df[comp] - df['Risk-Free'],
                  df[['Mkt-RF', 'SMB', 'HML', 'Mom   ']]).fit()
    params.append(reg2.params)
    print(reg2.summary2())

Company: ALLY

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.571   
Dependent Variable:     y                AIC:                         501.7471
Date:                   2020-05-28 22:05 BIC:                         514.4062
No. Observations:       175              Log-Likelihood:              -246.87 
Df Model:               4                F-statistic:                 59.14   
Df Residuals:           171              Prob (F-statistic):          2.84e-31
R-squared (uncentered): 0.580            Scale:                       1.0067  
-----------------------------------------------------------------------------------
             Coef.       Std.Err.         t         P>|t|        [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF       1.2569        0.1053      11.9347      0.0000       1.0491      1.4648
SMB         -0.0792        0.1961      -0


Company: BK

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.500   
Dependent Variable:     y                AIC:                         483.3724
Date:                   2020-05-28 22:05 BIC:                         496.0315
No. Observations:       175              Log-Likelihood:              -237.69 
Df Model:               4                F-statistic:                 44.72   
Df Residuals:           171              Prob (F-statistic):          1.16e-25
R-squared (uncentered): 0.511            Scale:                       0.90634 
-----------------------------------------------------------------------------------
             Coef.       Std.Err.         t         P>|t|        [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF       0.9262        0.0999       9.2686      0.0000       0.7290      1.1235
SMB         -0.0484        0.1860      -0.


Company: CPF

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.518   
Dependent Variable:     y                AIC:                         443.7317
Date:                   2020-05-28 22:05 BIC:                         456.3909
No. Observations:       175              Log-Likelihood:              -217.87 
Df Model:               4                F-statistic:                 48.11   
Df Residuals:           171              Prob (F-statistic):          4.69e-27
R-squared (uncentered): 0.529            Scale:                       0.72263 
-----------------------------------------------------------------------------------
              Coef.       Std.Err.        t         P>|t|        [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF        0.8182        0.0892      9.1700      0.0000       0.6421      0.9944
SMB           0.8586        0.1661      5


Company: GWB

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.712   
Dependent Variable:     y                AIC:                         460.5028
Date:                   2020-05-28 22:05 BIC:                         473.1620
No. Observations:       175              Log-Likelihood:              -226.25 
Df Model:               4                F-statistic:                 109.1   
Df Residuals:           171              Prob (F-statistic):          5.51e-46
R-squared (uncentered): 0.718            Scale:                       0.79531 
-----------------------------------------------------------------------------------
             Coef.       Std.Err.         t         P>|t|        [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF       1.3328        0.0936      14.2372      0.0000       1.1480      1.5175
SMB          0.8025        0.1743       4

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.784   
Dependent Variable:     y                AIC:                         399.3559
Date:                   2020-05-28 22:05 BIC:                         412.0151
No. Observations:       175              Log-Likelihood:              -195.68 
Df Model:               4                F-statistic:                 159.7   
Df Residuals:           171              Prob (F-statistic):          1.26e-56
R-squared (uncentered): 0.789            Scale:                       0.56078 
-----------------------------------------------------------------------------------
             Coef.       Std.Err.         t         P>|t|        [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF       1.2910        0.0786      16.4234      0.0000       1.1358      1.4461
SMB          0.2701        0.1463       1.8454      0.06


Company: ZBK

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): -0.014  
Dependent Variable:     y                AIC:                         363.7794
Date:                   2020-05-28 22:05 BIC:                         376.4385
No. Observations:       175              Log-Likelihood:              -177.89 
Df Model:               4                F-statistic:                 0.3812  
Df Residuals:           171              Prob (F-statistic):          0.822   
R-squared (uncentered): 0.009            Scale:                       0.45761 
-----------------------------------------------------------------------------------
             Coef.       Std.Err.         t         P>|t|        [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF      -0.0415        0.0710      -0.5849      0.5594      -0.1817      0.0986
SMB          0.0461        0.1322       0


Company: TD

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.488   
Dependent Variable:     y                AIC:                         308.4127
Date:                   2020-05-28 22:05 BIC:                         321.0719
No. Observations:       175              Log-Likelihood:              -150.21 
Df Model:               4                F-statistic:                 42.69   
Df Residuals:           171              Prob (F-statistic):          8.47e-25
R-squared (uncentered): 0.500            Scale:                       0.33350 
----------------------------------------------------------------------------------
                Coef.      Std.Err.        t        P>|t|       [0.025      0.975]
----------------------------------------------------------------------------------
Mkt-RF          0.6515       0.0606     10.7476     0.0000      0.5318      0.7712
SMB            -0.3357       0.1129     -2.974


Company: OFG

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.485   
Dependent Variable:     y                AIC:                         670.6594
Date:                   2020-05-28 22:05 BIC:                         683.3186
No. Observations:       175              Log-Likelihood:              -331.33 
Df Model:               4                F-statistic:                 42.15   
Df Residuals:           171              Prob (F-statistic):          1.45e-24
R-squared (uncentered): 0.496            Scale:                       2.6429  
-----------------------------------------------------------------------------------
               Coef.       Std.Err.        t         P>|t|       [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF         1.4921        0.1706      8.7436      0.0000      1.1552      1.8289
SMB            2.3837        0.3177      


Company: BBVA

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.464   
Dependent Variable:     y                AIC:                         537.2884
Date:                   2020-05-28 22:05 BIC:                         549.9476
No. Observations:       175              Log-Likelihood:              -264.64 
Df Model:               4                F-statistic:                 38.93   
Df Residuals:           171              Prob (F-statistic):          3.81e-23
R-squared (uncentered): 0.477            Scale:                       1.2334  
-----------------------------------------------------------------------------------
             Coef.       Std.Err.         t         P>|t|        [0.025      0.975]
-----------------------------------------------------------------------------------
Mkt-RF       1.0366        0.1166       8.8924      0.0000       0.8065      1.2667
SMB         -0.3582        0.2170      -